In [1]:
import pyspark
import os
import math
import random
import sys

# make sure pyspark tells workers to use python3 not 2 if both are installed\n",
os.environ["PYSPARK_PYTHON"] = "python"
spark_home = os.environ.get('SPARK_HOME', None)
print ( spark_home )
sc = pyspark.SparkContext('local[*]')
sc.setLogLevel('ERROR')
sc

None


<SparkContext master=local[*] appName=pyspark-shell>

In [40]:
import pandas as pd
df = pd.read_csv("./Groceries_dataset.csv")

In [41]:
df.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


Añadimos la columna transaction, donde todos los items que tengan el mismo Member_number y Date. Después eliminamos los registros duplicados del dataframe ya que no necesitamos la columna itemDescription. 

In [110]:
grouped =  df.groupby(['Member_number', 'Date']).agg(lambda x: ','.join(x)).reset_index()
grouped['itemDescription'] = grouped['itemDescription'].str.split(',')

grouped['itemDescription'] = grouped['itemDescription'].astype(str).str.replace('[', '').str.replace(']', '').str.replace('\'', '')

grouped.head()


KeyError: 'YourColumnName'

Transformamos la columna de transacciones a una lista separando cada item

In [ ]:
items_list = grouped['itemDescription'].str.split(', ').tolist()
print(items_list)

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,

In [ ]:
session = pyspark.sql.SparkSession(sc)
rddT =  sc.parallelize(items_list)
rddT.collect()

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [ ]:
# Compute the rdd with frequent singleton sets (L_1)
def computeL1 ( rddT, numtrans, theta ):
  rddtemp = rddT.flatMap( lambda t : [ (it,1) for it in t ] ).reduceByKey( lambda a,b : a+b  )
  return rddtemp.filter( lambda x : (float(x[1])/numtrans) >= theta )

In [ ]:
# Map any transaction to its version without elements not in L1
# L1 must be a python list, not a RDD
def computeTfilteredByL1( seqOfT, L1 ):
    for t in seqOfT:
       yield [ it for it in t if (it in L1) ]

Phase 2

In [ ]:
# For each t in seqofFilteredT (they come from T_{L_1}), compute pairs (a,b) from t that belong to C_2
def generateC2( seqofFilteredT ):
    for t in seqofFilteredT:
      cpairslist = []
      for (a,b) in [ (a,b) for i,a in enumerate(t[:-1]) for b in t[i+1:] ]:
                cpairslist.append( ((a,b),1) if (a <= b) else ((b,a),1)  )         
      yield cpairslist

Phase 3

In [ ]:
def computeL2( rddC2T, numtrans, theta ):
    pairsCountedrdd = rddC2T.reduceByKey( lambda v1,v2 : v1+v2 )
    # Finally, filter out from the previous rdd those pairs with frequency below theta
    return pairsCountedrdd.filter( lambda x : (float(x[1])/numtrans) >= theta )

In [ ]:
len(items_list)

14963

0.01, 0.1, 0.15

In [ ]:
numtrans = len(items_list)
theta= 0.009 
rddL1 = computeL1(rddT, numtrans, theta)
print(rddT)

ParallelCollectionRDD[57] at readRDDFromFile at PythonRDD.scala:287


In [ ]:
L1 = rddL1.keys().collect() # we need only the items (keys) in final L1
TL1 = rddT.mapPartitions( lambda seqOfT : computeTfilteredByL1( seqOfT, L1 )  )

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 9 in stage 101.0 failed 1 times, most recent failure: Lost task 9.0 in stage 101.0 (TID 369) (192.168.1.41 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\strix\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 830, in main
  File "C:\Users\strix\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 820, in process
  File "c:\Users\strix\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\rdd.py", line 5405, in pipeline_func
    return func(split, prev_func(split, iterator))
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\strix\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspar

In [ ]:
print(" L1 items: ", L1)
print(" Transactions with only frequent elements: ", TL1.collect())

 L1 items:  ['dtype: object']
 Transactions with only frequent elements:  [['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: object'], ['dtype: obje

In [ ]:
rddC2T = TL1.mapPartitions( lambda seqOfFilteredT : generateC2( seqOfFilteredT ) )
rddC2TFlat = rddC2T.flatMap( lambda x : x )

In [ ]:
print( "flattened C2T: ", rddC2TFlat.collect() )

flattened C2T:  []


In [ ]:
rddL2 = computeL2( rddC2TFlat, numtrans, theta )

In [ ]:
rddL2 = rddL2.sortBy(lambda a: -a[1])
for it in rddL2.toLocalIterator():
    print (it)


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "c:\Users\strix\AppData\Local\Programs\Python\Python311\Lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\strix\AppData\Local\Programs\Python\Python311\Lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\strix\AppData\Local\Programs\Python\Python311\Lib\socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 